In [1]:
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import numpy as np

In [2]:
train_df = pd.read_csv('./data/train.csv')
train_df.head()

,x,y,color,marker
0,-0.765775,1.006452,red,^
1,0.229016,0.094320,red,^
2,0.189987,1.589141,red,^
3,0.580085,0.520488,red,^
4,0.292287,0.317852,red,^


In [3]:
# check the different color and marker options
color_list = train_df.color.unique()
print(train_df.color.unique())
marker_list = train_df.marker.unique()
print(train_df.marker.unique())
output_list = np.concatenate((color_list, marker_list), axis = 0)
print(output_list)

['red' 'blue' 'green' 'teal' 'orange' 'purple']
['^' '+' '*']
['red' 'blue' 'green' 'teal' 'orange' 'purple' '^' '+' '*']


In [4]:
# convert the colors and markers into numbers in hot format: eg. [0 0 0 0 0 1 0 1 0] (colors first 5, markers last 3)
one_hot_color = pd.get_dummies(train_df.color).values
one_hot_marker = pd.get_dummies(train_df.marker).values
train_output_values = np.concatenate((one_hot_color, one_hot_marker), axis = 1)
print(train_output_values[0:5])

[[0 0 0 0 1 0 0 0 1]
 [0 0 0 0 1 0 0 0 1]
 [0 0 0 0 1 0 0 0 1]
 [0 0 0 0 1 0 0 0 1]
 [0 0 0 0 1 0 0 0 1]]


Convert dataframe into numpy array to be used in the network

In [5]:
train_input_values = np.column_stack((train_df.x.values, train_df.y.values))

# shuffle the data, ensuring input and output are shuffled in sync
np.random.RandomState(seed = 69).shuffle(train_input_values)
np.random.RandomState(seed = 69).shuffle(train_output_values)

print("x and y values:\n", train_input_values[0:5])
print("\ncolor and marker hot values:\n",train_output_values[0:5])

x and y values:
 [[-2.38000522  2.70663374]
 [-0.82326915  4.45782714]
 [-1.02869568  2.34328854]
 [-0.66852994  2.68389823]
 [-2.885875    2.44154295]]

color and marker hot values:
 [[0 0 1 0 0 0 0 1 0]
 [1 0 0 0 0 0 0 0 1]
 [0 0 1 0 0 0 0 1 0]
 [0 0 0 1 0 0 1 0 0]
 [0 0 1 0 0 0 0 1 0]]


In [6]:
model = keras.Sequential([
    keras.layers.Dense(64, input_shape = (2,), activation = 'relu'), # 64 neurons in our first hidden layer
    keras.layers.Dense(64, activation = 'relu'), # add another layer
    keras.layers.Dense(9, activation = 'sigmoid')]) # output layer of 9, representing the colors and markers

model.compile(optimizer = 'adam',
              loss = keras.losses.BinaryCrossentropy(from_logits = True), # Get rid of Sparse
              metrics = ['accuracy'])

model.fit(train_input_values, train_output_values, batch_size = 4, epochs = 10)

Train on 6000 samples
Epoch 1/10
6000/6000 [==============================] - 3s 418us/sample - loss: 0.6726 - accuracy: 0.8492
Epoch 2/10
6000/6000 [==============================] - 2s 344us/sample - loss: 0.6563 - accuracy: 0.8789
Epoch 3/10
6000/6000 [==============================] - 2s 373us/sample - loss: 0.6548 - accuracy: 0.8822
Epoch 4/10
6000/6000 [==============================] - 2s 382us/sample - loss: 0.6539 - accuracy: 0.8835
Epoch 5/10
6000/6000 [==============================] - 2s 382us/sample - loss: 0.6499 - accuracy: 0.8957
Epoch 6/10
6000/6000 [==============================] - 2s 366us/sample - loss: 0.6450 - accuracy: 0.9101
Epoch 7/10
6000/6000 [==============================] - 3s 425us/sample - loss: 0.6419 - accuracy: 0.9171
Epoch 8/10
6000/6000 [==============================] - 2s 355us/sample - loss: 0.6366 - accuracy: 0.9322
Epoch 9/10
6000/6000 [==============================] - 2s 356us/sample - loss: 0.6352 - accuracy: 0.9350
Epoch 10/10
6000/6000 [=

Now we use our model on the test data

In [7]:
test_df = pd.read_csv('./data/test.csv')
test_input_values = np.column_stack((test_df.x.values, test_df.y.values))

# convert the colors and markers to ints
test_one_hot_color = pd.get_dummies(test_df.color).values
test_one_hot_marker = pd.get_dummies(test_df.marker).values
test_output_values = np.concatenate((test_one_hot_color, test_one_hot_marker), axis = 1)

test_df.head()

,x,y,color,marker
0,-0.129697,0.773598,red,^
1,0.341569,0.505299,red,^
2,0.168562,1.397924,red,^
3,-0.738458,0.097241,red,^
4,0.311101,1.669878,red,^


In [8]:
model.evaluate(test_input_values, test_output_values)

1200/1200 [==============================] - 0s 159us/sample - loss: 0.6351 - accuracy: 0.9331


[0.6350663216908773, 0.9330555]

In [11]:
# test a co-ordinate to see if it gives us the correct output
print(output_list)
print(np.round(model.predict(np.array([[0, 3], [0, 1], [-2, 1]]))))

['red' 'blue' 'green' 'teal' 'orange' 'purple' '^' '+' '*']
[[0. 0. 0. 1. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 1.]
 [0. 1. 0. 0. 0. 0. 0. 1. 0.]]
